In [1]:
import pandas as pd
import os
import glob

Create data_silo's 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

folder_path = "/proj/sourasb-220503/FedMEM/dataset/clients/data_silo/A1/100"
new_path = "/proj/sourasb-220503/FedMEM/dataset/clients/data_silo/A1/20"
data_silo=0.2
# Load the dataset

for filename in os.listdir(folder_path):
    if filename.endswith('training.csv'):
        file_path = os.path.join(folder_path, filename)
        train_df = pd.read_csv(file_path, header=None)

        dir_name = filename[:12]
        target_path = new_path + "/" + dir_name
        if not os.path.exists(target_path):
            os.makedirs(target_path)
            print(f"Created directory: {target_path}")


        # Assuming train_df is your initial DataFrame
        num_samples = int(len(train_df)*data_silo)  # 50% of the total data

        dataframes = []  # This will store your 5 DataFrames

        for i in range(5):
            # Shuffle train_df
            shuffled_df = shuffle(train_df, random_state=None)  # Random state None for different shuffle each time
            
            # Take the first 50% of the shuffled DataFrame
            subset_df = shuffled_df.iloc[:num_samples]
            
            # Add the subset to your list of DataFrames
            dataframes.append(subset_df)
            
            # Define a file name for the current DataFrame
            file_name = f"Client_ID_training_{i}.csv"
            file_path = os.path.join(target_path, file_name)
            # Save the DataFrame to a CSV file
            subset_df.to_csv(file_path, index=False, header=False)
            
            print(f"Saved DataFrame {i} to {file_path}")
    


To copy the validation set to all data silo


In [ ]:
import shutil

source_path = "/proj/sourasb-220503/FedMEM/dataset/clients/data_silo/A1/100"
new_path = "/proj/sourasb-220503/FedMEM/dataset/clients/data_silo/A1/20"

# Copy the file

for filename in os.listdir(folder_path):
    if filename.endswith('validation.csv'):
        file_path = os.path.join(source_path, filename)

        dir_name = filename[:12]
        target_path = new_path + "/" + dir_name 
        if not os.path.exists(target_path):
            os.makedirs(target_path)
            print(f"Created directory: {target_path}")
        destination_path = target_path + "/" +filename
        shutil.copy(file_path, target_path)
        print(f"File copied from {source_path} to {destination_path}")

In [16]:
# The path to the directory containing your CSV files
input_directory = '/proj/sourasb-220503/FedMEM/dataset/clients/A1'
# The path to the directory where you want to save the new CSV files
output_directory = '/proj/sourasb-220503/FedMEM/dataset/clients/data_silo/A1/20'

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# List all CSV files in the input directory
csv_files = glob.glob(os.path.join(input_directory, '*.csv'))

# Process each file
for file_path in csv_files:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Select 20% of the rows sequentially
    num_rows = len(df)
    subset_df = df.iloc[:int(num_rows * 0.2)]

    # Define the new file path in the output directory
    # Extract the file name from the original path and prepend the output directory path
    file_name = os.path.basename(file_path)
    new_file_path = os.path.join(output_directory, file_name)

    # Save the new DataFrame to a CSV file in the output directory
    subset_df.to_csv(new_file_path, index=False)

print("Processing completed. The files have been saved to the '20' directory.")

Processing completed. The files have been saved to the '20' directory.


In [51]:
# The path to the directory where the new CSV files are saved
directory = '/proj/sourasb-220503/FedMEM/dataset/clients/data_silo/A1/20/Client_ID_16'

# List all CSV files in the output directory
csv_files = glob.glob(os.path.join(directory, '*.csv'))

# Process each file
for file_path in csv_files:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Get the number of rows in the DataFrame
    num_rows = len(df)

    # Print the file name and the number of rows it contains
    print(f'File: {os.path.basename(file_path)} - Rows: {num_rows}')

File: Client_ID_training_2.csv - Rows: 43
File: Client_ID_training_3.csv - Rows: 43
File: Client_ID_training_0.csv - Rows: 43
File: Client_ID_training_1.csv - Rows: 43
File: Client_ID_16_validation.csv - Rows: 55
File: Client_ID_training_4.csv - Rows: 43


Find identical rows accross all columns


In [63]:
import pandas as pd
import numpy as np
import glob
import os

directory = '/proj/sourasb-220503/FedMEM/dataset/clients/data_silo/A1/80/Client_ID_23'  # Adjust this to your directory

# List all CSV files in the directory

# List all CSV files in the directory
csv_files = glob.glob(os.path.join(directory, '*.csv'))
file_names = [os.path.basename(f) for f in csv_files]

overlap_matrix = pd.DataFrame(np.zeros((len(csv_files), len(csv_files))), columns=file_names, index=file_names)


# Compare each file with every other file in the list
for i in range(len(csv_files)):
    for j in range(i, len(csv_files)):
        df1 = pd.read_csv(csv_files[i], names=['file', 'score'])
        df2 = pd.read_csv(csv_files[j], names=['file', 'score'])

        # Find overlapping rows
        overlap = pd.merge(df1, df2, how='inner')

        overlap_count = len(overlap)

        # Print the number of overlapping rows
        #print(f'Overlap between {os.path.basename(csv_files[i])} and {os.path.basename(csv_files[j])}: {len(overlap)} rows')
        overlap_matrix.iloc[i, j] = overlap_count
        overlap_matrix.iloc[j, i] = overlap_count  # Mirror the count since the matrix is symmetric

# Display the overlap matrix
overlap_matrix

,Client_ID_training_2.csv,Client_ID_training_3.csv,Client_ID_training_0.csv,Client_ID_training_1.csv,Client_ID_23_validation.csv,Client_ID_training_4.csv
Client_ID_training_2.csv,173.0,142.0,136.0,137.0,0.0,140.0
Client_ID_training_3.csv,142.0,173.0,141.0,134.0,0.0,138.0
Client_ID_training_0.csv,136.0,141.0,173.0,138.0,0.0,134.0
Client_ID_training_1.csv,137.0,134.0,138.0,173.0,0.0,138.0
Client_ID_23_validation.csv,0.0,0.0,0.0,0.0,55.0,0.0
Client_ID_training_4.csv,140.0,138.0,134.0,138.0,0.0,173.0
